# Run this jupyter notebook to get hourly average prices (LMP, MCC, MCE, MCL) for a given time range

In [1]:
import os
from glob import glob
import pandas as pd
import sys

sys.path.append(os.getcwd())

## Scrape LMP (along with MCL, MCE, MCC) in DAM

In [ ]:
# replace the following variables with your own
_market = "DAM"
_start_date = "20230723"
_end_date = "20230727"

data_folder = "data/LMP" #just updated as LMP, instead of prices
zip_data_folder = os.path.join(data_folder, "raw")

In [ ]:
%run -i 'scraper_CAISO_LMP.py' --market {_market} --start_date {_start_date} --end_date {_end_date} --save_to_path {zip_data_folder}

## Unzip files

In [ ]:
from utils import unzip_files

unzip_data_folder = os.path.join(data_folder, 'unzip')
unzip_files(input_dir=zip_data_folder, output_dir=unzip_data_folder)

## Prepare LMP, MCC, MCE, MCL files

In [ ]:
# load LMP csv files
current_dir = os.getcwd() # use os.path.dirname(__file__) does not work in jupyter notebook
data_dir = os.path.join(current_dir, unzip_data_folder)

LMP_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_LMP_*.csv')))
MCC_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_MCC_*.csv')))
MCE_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_MCE_*.csv')))
MCL_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_MCL_*.csv')))

print("===LMP===\n")
print([os.path.basename(file) for file in LMP_csv_files[:5]], "\n")
print("===MCC===\n")
print([os.path.basename(file) for file in MCC_csv_files[:5]], "\n")
print("===MCE===\n")
print([os.path.basename(file) for file in MCE_csv_files[:5]], "\n")
print("===MCL===\n")
print([os.path.basename(file) for file in MCL_csv_files[:5]], "\n")

## Calculate hourly average price for LMP, MCC, MCE, MCL and save the results into separate csv files

In [ ]:
from draw_avg_price import find_hourly_avg_price

avg_price_folder = os.path.join(data_folder, 'avg_hourly_price')

find_hourly_avg_price(
    LMP_csv_files=LMP_csv_files,
    MCC_csv_files=MCC_csv_files,
    MCE_csv_files=MCE_csv_files,
    MCL_csv_files=MCL_csv_files,
    output_dir=avg_price_folder,
)

print('Done.')

In [ ]:
sorted(glob(os.path.join(avg_price_folder, '*')))

## Peek average prices

In [ ]:
for file in sorted(glob(os.path.join(current_dir, avg_price_folder, '*'))):
    df = pd.read_csv(file, low_memory=False)
    print(df.head())

In [2]:
from draw_avg_price import draw_avg_price_for_multiple_days

data_folder = "data/LMP"
avg_price_folder = os.path.join(data_folder, 'avg_hourly_price_0723_0823')
draw_avg_price_for_multiple_days(avg_price_folder)